In [49]:
import json
import pandas as pd
import numpy as np
import requests
import csv
import geopandas as gpd
import  re
import chardet
import shapely
from shapely.geometry import Polygon
from shapely.geometry import Point
import h3
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
from geopy.geocoders import Nominatim
import uuid

In [ ]:
datasets_nums = [619, 2781, 611, 526, 493, 629, 624, 61745]
dataset_names = ["kiosks", "papers_kiosks", "gosuslugi", "libraries", "culture_clubs", "sport", "metro_entrances", 'districts']
datasets = []
keys = [[],
        [],
        [],
        [],
        [],
        []]

for i, code in enumerate(datasets_nums):
    response = requests.post(url=f'https://apidata.mos.ru/v1/datasets/{code}/features?api_key=ac106007dd68b52c51ae1062883b8187')
    datasets.append(response.json())

In [ ]:
for i, name in enumerate(dataset_names):
    jsondata = datasets[i]['features']

    data_file = open(f'{name}.csv', 'w', newline='', encoding="utf-8")
    csv_writer = csv.writer(data_file)

    count = 0
    for data in jsondata:
        keys = []
        values = []
        for key in data.keys():
            val = data[key]
            if type(val) != type(data):
                continue

            for key1 in val.keys():
                if type(val[key1]) != type(val):
                    continue
                values += val[key1].values()
                keys += val[key1].keys()

            values += data[key].values()
            keys += data[key].keys()

        if count == 0:
            header = keys
            csv_writer.writerow(header)
            count += 1
        csv_writer.writerow(values)

    data_file.close()

In [50]:
metro_total = pd.read_excel('metro_stats.xlsx').drop(0)

In [51]:
with open("data-397-2022-10-21.json", 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']

metro_entrances = pd.read_json("data-397-2022-10-21.json", encoding=encoding)

def get_name(full_name):
    full_name = full_name.lower()
    full_name = re.sub("ё", "е", full_name)
    full_name = full_name.split(", ")[0]
    for i in ["народное ополчение", "мневники", "беговая", "электрозаводская", "киевская", "румянцево", "арбатская"]:
        if full_name.startswith(i):
            return i

    if full_name.startswith("театральная"):
        return "Охотный ряд".lower()
    if full_name.startswith("библиотека имени ленина"):
        return "боровицкая".lower()
    if full_name.startswith("улица старокачаловская"):
        return "Бульвар Дмитрия Донского".lower()

    return full_name

metro_entrances["Name"] = metro_entrances["Name"].apply(get_name)
metro_total["Name"] = metro_total["NameOfStation"].apply(get_name)
metro_entrances = metro_entrances[["Name", "Latitude_WGS84", "Longitude_WGS84"]]

metro_entrances = metro_entrances.groupby(['Name']).mean()

metro_total['IncomingPassengers'] = metro_total['IncomingPassengers'].astype(int)
metro_total['OutgoingPassengers'] = metro_total['OutgoingPassengers'].astype(int)
metro_total['TotalPassengers'] = metro_total['OutgoingPassengers'] + metro_total['IncomingPassengers']

metro_total = metro_total.groupby(['Name']).mean()

In [52]:
metro_total = metro_total.join(metro_entrances, on='Name')

In [53]:
RESOLUTION = 8

def get_h3(lng, lat, res=RESOLUTION):
    row = pd.Series()
    row['h3'] = h3.geo_to_h3(lat=lat, lng=lng, resolution=res)
    return row

h3_Series = metro_total.apply(lambda x: get_h3(lat=x['Latitude_WGS84'],
                                               lng=x['Longitude_WGS84'],
                                               res=RESOLUTION), axis=1)
metro_total = metro_total.join(h3_Series, on='Name')

names = metro_total.index
metro_total.set_index('h3', inplace=True)
metro_total['Name'] = names

/var/folders/hd/xt4njp311pn0fwcx30kcfr6r0000gp/T/ipykernel_88124/839195815.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  row = pd.Series()
/var/folders/hd/xt4njp311pn0fwcx30kcfr6r0000gp/T/ipykernel_88124/839195815.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  row = pd.Series()
/var/folders/hd/xt4njp311pn0fwcx30kcfr6r0000gp/T/ipykernel_88124/839195815.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  row = pd.Series()
/var/folders/hd/xt4njp311pn0fwcx30kcfr6r0000gp/T/ipykernel_88124/839195815.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicit

,IncomingPassengers,OutgoingPassengers,TotalPassengers,Latitude_WGS84,Longitude_WGS84,Name
h3,,,,,,
8811aa7897fffff,2.007669e+06,2.093297e+06,4.100966e+06,55.751692,37.717328,авиамоторная
8811aa79d9fffff,1.700653e+06,1.815051e+06,3.515704e+06,55.706967,37.659275,автозаводская
8811aa7869fffff,2.657332e+06,2.671209e+06,5.328541e+06,55.687789,37.573498,академическая
8811aa7aabfffff,1.278142e+06,1.286268e+06,2.564410e+06,55.752193,37.610628,александровский сад
8811aa634bfffff,2.620293e+06,2.699040e+06,5.319333e+06,55.807929,37.638540,алексеевская
...,...,...,...,...,...,...
8811aa6323fffff,1.718966e+06,1.806569e+06,3.525536e+06,55.781537,37.704738,электрозаводская
8811aa6a37fffff,8.646050e+05,8.242832e+05,1.688888e+06,55.705605,37.817261,юго-восточная
8811aa7b45fffff,3.734187e+06,3.757222e+06,7.491409e+06,55.663670,37.483204,юго-западная


In [55]:
pop_loc = "pop.gpkg"
pop = gpd.read_file(pop_loc)
metro = pd.read_excel('metro_total.xlsx')
pop.set_index(['h3'], inplace=True)
metro.set_index(['h3'], inplace=True)
metro.join(pop, on='h3').to_excel('metro_pop.xlsx')

In [57]:
districts = pd.read_csv('districts.csv')

df = pd.read_excel('datapoints_coor.xlsx')
df['district'] = pd.Series()
df['district'] = df['district'].fillna(value='none')

/var/folders/hd/xt4njp311pn0fwcx30kcfr6r0000gp/T/ipykernel_88124/678385603.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['district'] = pd.Series()


,Unnamed: 0,coordinates,NumOfReaders,NumOfVisitors,tag,h3,lat,lng,district
0,0,"[37.5920849999999, 55.73346]",NaN,NaN,kiosks,8911aa785afffff,55.733460,37.592085,none
1,1,"[37.582864, 55.726996]",NaN,NaN,kiosks,8911aa78513ffff,55.726996,37.582864,none
2,2,"[37.573840999, 55.720382]",NaN,NaN,kiosks,8911aa785d3ffff,55.720382,37.573841,none
3,3,"[37.6012429999999, 55.743848]",NaN,NaN,kiosks,8911aa7aa8bffff,55.743848,37.601243,none
4,4,"[37.640956, 55.807812]",NaN,NaN,kiosks,8911aa634a3ffff,55.807812,37.640956,none
...,...,...,...,...,...,...,...,...,...
5158,5158,"[[37.1552145989015, 55.9744720848914]]",NaN,NaN,sport,8911aa002c3ffff,55.974472,37.155215,none
5159,5159,"[[37.1742207686289, 55.9628852126472]]",NaN,NaN,sport,8911aa0118fffff,55.962885,37.174221,none
5160,5160,"[[37.713616964831, 55.6149512075357]]",NaN,NaN,sport,891181b6bd3ffff,55.614951,37.713617,none
5161,5161,"[[37.4562317085733, 55.7347606005486]]",NaN,NaN,sport,8911aa44a2bffff,55.734761,37.456232,none


In [58]:
def format_dist(s):
    import re
    try:
        s = s.lower()
    except Exception:
        return 'none'
    s = re.sub('район', '', s)
    s = re.sub('городской округ', '', s)
    s = re.sub(r'[^\w]', '', s)
    s = re.sub(' ', '', s)
    return s


districts['AuthorityArea'] = districts['AuthorityArea'].apply(lambda x: format_dist(x))
df['district'] = df['district'].apply(lambda x: format_dist(x))
nan_coords = df[df['district'] == 'none'][['lat', 'lng']]

,lat,lng
0,55.733460,37.592085
1,55.726996,37.582864
2,55.720382,37.573841
3,55.743848,37.601243
4,55.807812,37.640956
...,...,...
5158,55.974472,37.155215
5159,55.962885,37.174221
5160,55.614951,37.713617
5161,55.734761,37.456232


In [59]:
geolocator = Nominatim(user_agent="user_agent")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1/100)

for idx in tqdm(nan_coords.index):
    lat = float(nan_coords[nan_coords.index==idx]['lat'])
    lng = float(nan_coords[nan_coords.index==idx]['lng'])

    location = reverse([lat, lng], addressdetails=True)
    suburb = location.raw["address"].get("suburb")
    df['district'][idx] = suburb

  0%|          | 0/5163 [00:00<?, ?it/s]/var/folders/hd/xt4njp311pn0fwcx30kcfr6r0000gp/T/ipykernel_88124/629956255.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['district'][idx] = suburb
 28%|██▊       | 1425/5163 [12:00<31:30,  1.98it/s]


KeyboardInterrupt: 

In [ ]:
df['district'] = df['district'].apply(lambda x: format_dist(x))
nan_coords = df[df['district'] == 'none'][['lat', 'lng']]

for idx in tqdm(nan_coords.index):
    lat = float(nan_coords[nan_coords.index==idx]['lat'])
    lng = float(nan_coords[nan_coords.index==idx]['lng'])

    location = reverse([lat, lng], addressdetails=True)
    suburb = location.raw["address"].get("town")
    df['district'][idx] = suburb

In [ ]:
df['district'] = df['district'].apply(lambda x: format_dist(x))
nan_coords = df[df['district'] == 'none'][['lat', 'lng']]

for idx in tqdm(nan_coords.index):
    lat = float(nan_coords[nan_coords.index==idx]['lat'])
    lng = float(nan_coords[nan_coords.index==idx]['lng'])

    location = reverse([lat, lng], addressdetails=True)
    suburb = location.raw["address"].get("village")
    df['district'][idx] = suburb

In [ ]:
df['district'] = df['district'].apply(lambda x: format_dist(x))
nan_coords = df[df['district'] == 'none'][['lat', 'lng']]

for idx in tqdm(nan_coords.index):
    lat = float(nan_coords[nan_coords.index==idx]['lat'])
    lng = float(nan_coords[nan_coords.index==idx]['lng'])

    location = reverse([lat, lng], addressdetails=True)
    suburb = location.raw["address"].get("municipality")
    df['district'][idx] = suburb

In [ ]:
df['district'] = df['district'].apply(lambda x: format_dist(x))
out = df.merge(districts, left_on='district', right_on='AuthorityArea', how='outer')
df = out

In [ ]:
df['address'] = pd.Series()
for idx in tqdm(df.index):
    lat = float(df[df.index==idx]['lat'])
    lng = float(df[df.index==idx]['lng'])

    location = reverse([lat, lng], addressdetails=True)
    address = 'None'

    if location.raw["address"].get('road') != None:
        address = location.raw["address"].get('road')

        if location.raw["address"].get('house_number') != None:
            address +=  ', ' + 'д. ' + location.raw["address"].get('house_number')
    df['address'][idx] = address

In [ ]:
df.to_excel('datapoints_districts_total.xlsx')

mongoose_datapoints = df.dropna(subset=['AdmArea_y'])[df['AdmArea_y'] != 'Зеленоградский административный округ'][
    ['address', 'tag', 'district', 'AdmArea_y', 'lat', 'lng']]
mongoose_datapoints.columns = ['address', 'type', 'district', 'adminstrativeDistrict', 'lat', 'lon']
mongoose_datapoints.to_excel('mongoose_datapoints.xlsx')